In [1]:
import pandas as pd

from Bio import PDB

# 3->1-betűs aminosav-kód konverzió dictionary
from Bio.Data.IUPACData import protein_letters_3to1

In [2]:
pfam = pd.read_csv('/home/takacsg/Documents/sh2db/data/SH2_domain_containing_prot_right_resnum_with_pdb_nums_domcol.csv')
pfam.head()

,Unnamed: 0,UniProt entry,Gene_domain,Pole,UniProt residues,PDB ID,PDB chain ID,PDB residues,Category,Gene name,...,Res_stop,Uniprot_ID,New_uniprot_start,New_uniprot_stop,Old_uniprot_start,Old_uniprot_stop,diff_start,New_PDB_start,diff_stop,New_PDB_stop
0,0,3BP2_HUMAN,SH3BP2,N,458 - 538,2CR4,A,20 - 100,SigReg,SH3BP2,...,100,P78314,435,561,458,538,23,0,23,123
1,1,ABL1_HUMAN,ABL1,N,127 - 202,1AB2,A,11 - 86,Kinases,ABL1,...,86,P00519,105,228,127,202,22,0,26,112
2,2,ABL1_HUMAN,ABL1,N,127 - 202,1OPL,A,146 - 221,Kinases,ABL1,...,221,P00519,105,228,127,202,22,124,26,247
3,3,ABL1_HUMAN,ABL1,N,127 - 202,1OPL,B,146 - 221,Kinases,ABL1,...,221,P00519,105,228,127,202,22,124,26,247
4,4,ABL1_HUMAN,ABL1,N,127 - 202,2ABL,A,146 - 221,Kinases,ABL1,...,221,P00519,105,228,127,202,22,124,26,247


In [14]:
parser = PDB.PDBParser(QUIET=True)
x = 0

for cat in pfam["Category"].unique():
    for gene in pfam[pfam["Category"] == cat]["Gene name"].unique():
        for pdb in pfam[pfam["Gene name"] == gene]["PDB ID"].unique():
            for chain in pfam[pfam["PDB ID"] == pdb]["PDB chain ID"].unique():
                for pole in pfam[(pfam["PDB ID"] == pdb) & (pfam["PDB chain ID"] == chain)]["Pole"].unique():
                    pdb_id = str(pdb+'_'+chain+'_SH2'+pole)                    
                    try:
                        #print(pdb_id)
                        structure = parser.get_structure(pdb_id, '/home/takacsg/Documents/sh2db/Structures/'+cat+'/'+gene+'/'+pdb+'/'+pdb+'_'+chain+'_SH2_'+pole+'.pdb') # struct delles pdb-fájlokat nem szereti!
                    except ValueError:
                        continue
                    #model = structure[0]
                    residues = PDB.Selection.unfold_entities(structure, "R")
                    chain = (pdb+'_'+chain+'_SH2'+pole).split('_')[1]
                    """for residue in residues:
                        if residue in ['Hoh']: #This can be expanded if multiple problematic residues occur
                            residues.remove(residue) # dropping key in the next line (for "Hoh")
                    print(residues)"""
                    try:
                        pdbseq = list(zip([protein_letters_3to1[i.get_resname().title()] for i in residues],[i.get_id()[1] for i in residues]))
                    except KeyError: #Hoh caused the Keyerror problem but was hard to remove
                        continue
                    if x % 1 == 0:
                        print("PDB ID", pdb_id, "\n", "CHAIM", chain, "\n", "PDBSEQ", pdbseq)

PDB ID 2CR4_A_SH2N 
 CHAIM A 
 PDBSEQ [('G', 1), ('S', 2), ('S', 3), ('G', 4), ('S', 5), ('S', 6), ('G', 7), ('E', 8), ('D', 9), ('Y', 10), ('E', 11), ('K', 12), ('V', 13), ('P', 14), ('L', 15), ('P', 16), ('N', 17), ('S', 18), ('V', 19), ('F', 20), ('V', 21), ('N', 22), ('T', 23), ('T', 24), ('E', 25), ('S', 26), ('C', 27), ('E', 28), ('V', 29), ('E', 30), ('R', 31), ('L', 32), ('F', 33), ('K', 34), ('A', 35), ('T', 36), ('S', 37), ('P', 38), ('R', 39), ('G', 40), ('E', 41), ('P', 42), ('Q', 43), ('D', 44), ('G', 45), ('L', 46), ('Y', 47), ('C', 48), ('I', 49), ('R', 50), ('N', 51), ('S', 52), ('S', 53), ('T', 54), ('K', 55), ('S', 56), ('G', 57), ('K', 58), ('V', 59), ('L', 60), ('V', 61), ('V', 62), ('W', 63), ('D', 64), ('E', 65), ('T', 66), ('S', 67), ('N', 68), ('K', 69), ('V', 70), ('R', 71), ('N', 72), ('Y', 73), ('R', 74), ('I', 75), ('F', 76), ('E', 77), ('K', 78), ('D', 79), ('S', 80), ('K', 81), ('F', 82), ('Y', 83), ('L', 84), ('E', 85), ('G', 86), ('E', 87), ('V', 88), ('

In [4]:
table = pd.read_csv('/home/takacsg/Documents/sh2db/data/proba.csv')
table.head()

,0,1,2,3,4,5,6,7,8,9,...,241,242,243,244,245,246,247,248,249,250
0,STAT5A,seq,-,-,-,-,-,-,N,Y,...,A,D,A,G,G,S,S,A,T,Y
1,STAT5A,nums,-,-,-,-,-,-,567,568,...,711,712,713,714,715,716,717,718,719,720
2,STAT5B,seq,-,-,-,-,G,R,N,Y,...,A,D,A,G,G,G,S,A,T,Y
3,STAT5B,nums,,,,,565,566,567,568,...,716,717,718,719,720,721,722,723,724,725
4,STAT6,seq,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


PDB ID 2CR4_A_SH2_N 
 CHAIN A 
 PDBSEQ [('G', 1), ('S', 2), ('S', 3), ('G', 4), ('S', 5), ('S', 6), ('G', 7), ('E', 8), ('D', 9), ('Y', 10), ('E', 11), ('K', 12), ('V', 13), ('P', 14), ('L', 15), ('P', 16), ('N', 17), ('S', 18), ('V', 19), ('F', 20), ('V', 21), ('N', 22), ('T', 23), ('T', 24), ('E', 25), ('S', 26), ('C', 27), ('E', 28), ('V', 29), ('E', 30), ('R', 31), ('L', 32), ('F', 33), ('K', 34), ('A', 35), ('T', 36), ('S', 37), ('P', 38), ('R', 39), ('G', 40), ('E', 41), ('P', 42), ('Q', 43), ('D', 44), ('G', 45), ('L', 46), ('Y', 47), ('C', 48), ('I', 49), ('R', 50), ('N', 51), ('S', 52), ('S', 53), ('T', 54), ('K', 55), ('S', 56), ('G', 57), ('K', 58), ('V', 59), ('L', 60), ('V', 61), ('V', 62), ('W', 63), ('D', 64), ('E', 65), ('T', 66), ('S', 67), ('N', 68), ('K', 69), ('V', 70), ('R', 71), ('N', 72), ('Y', 73), ('R', 74), ('I', 75), ('F', 76), ('E', 77), ('K', 78), ('D', 79), ('S', 80), ('K', 81), ('F', 82), ('Y', 83), ('L', 84), ('E', 85), ('G', 86), ('E', 87), ('V', 88), (

In [ ]:
parser = PDB.PDBParser(QUIET=True)
x = 0

for cat in pfam["Category"].unique():
    for gene in pfam[pfam["Category"] == cat]["Gene name"].unique():
        for pdb in pfam[pfam["Gene name"] == gene]["PDB ID"].unique():
            for chain in pfam[pfam["PDB ID"] == pdb]["PDB chain ID"].unique():
                for pole in pfam[(pfam["PDB ID"] == pdb) & (pfam["PDB chain ID"] == chain)]["Pole"].unique():
                    pdb_id = str(pdb+'_'+chain+'_SH2_'+pole)
                    try:
                        #print(pdb_id)
                        structure = parser.get_structure(pdb_id, '/home/takacsg/Documents/sh2db/Structures/'+cat+'/'+gene+'/'+pdb+'/'+pdb+'_'+chain+'_SH2_'+pole+'.pdb') # struct delles pdb-fájlokat nem szereti!
                    except ValueError:
                        continue
                    #model = structure[0]
                    residues = PDB.Selection.unfold_entities(structure, "R")
                    chain = pdb_id.split('_')[1]
                    """for residue in residues:
                        if residue in ['Hoh']: #This can be expanded if multiple problematic residues occur
                            residues.remove(residue) # dropping key in the next line (for "Hoh")
                    print(residues)"""
                    try:
                        pdbseq = list(zip([protein_letters_3to1[i.get_resname().title()] for i in residues],[i.get_id()[1] for i in residues]))
                    except KeyError: #Hoh caused the Keyerror problem but was hard to remove
                        continue
                    if x % 1 == 0:
                        print("PDB ID", pdb_id, "\n", "CHAIN", chain, "\n", "PDBSEQ", pdbseq)
                    increment = pfam[(pfam['PDB ID']==pdb) & (pfam['PDB chain ID']==chain)]['New_uniprot_stop'].astype(int) - pfam[(pfam['PDB ID']==pdb) & (pfam['PDB chain ID']==chain)]['New_PDB_stop'].astype(int)
                    # <- idáig jó - inentől nem -->
                    for (resname,resid) in pdbseq:
                        try:
                            s=table.loc[(gene ,'nums')]==str(resid+increment)
                            col=s[s].index[0]
                            table.loc[(gene, pdb_id),col]=resname
                        except KeyError:
                            continue

table.to_csv('../data/table_from_alignment_with_pdbs.csv')parser = PDB.PDBParser(QUIET=True)
x = 0

for cat in pfam["Category"].unique():
    for gene in pfam[pfam["Category"] == cat]["Gene name"].unique():
        for pdb in pfam[pfam["Gene name"] == gene]["PDB ID"].unique():
            for chain in pfam[pfam["PDB ID"] == pdb]["PDB chain ID"].unique():
                for pole in pfam[(pfam["PDB ID"] == pdb) & (pfam["PDB chain ID"] == chain)]["Pole"].unique():
                    pdb_id = str(pdb+'_'+chain+'_SH2_'+pole)
                    try:
                        #print(pdb_id)
                        structure = parser.get_structure(pdb_id, '/home/takacsg/Documents/sh2db/Structures/'+cat+'/'+gene+'/'+pdb+'/'+pdb+'_'+chain+'_SH2_'+pole+'.pdb') # struct delles pdb-fájlokat nem szereti!
                    except ValueError:
                        continue
                    #model = structure[0]
                    residues = PDB.Selection.unfold_entities(structure, "R")
                    chain = pdb_id.split('_')[1]
                    """for residue in residues:
                        if residue in ['Hoh']: #This can be expanded if multiple problematic residues occur
                            residues.remove(residue) # dropping key in the next line (for "Hoh")
                    print(residues)"""
                    try:
                        pdbseq = list(zip([protein_letters_3to1[i.get_resname().title()] for i in residues],[i.get_id()[1] for i in residues]))
                    except KeyError: #Hoh caused the Keyerror problem but was hard to remove
                        continue
                    if x % 1 == 0:
                        print("PDB ID", pdb_id, "\n", "CHAIN", chain, "\n", "PDBSEQ", pdbseq)
                    increment = pfam[(pfam['PDB ID']==pdb) & (pfam['PDB chain ID']==chain)]['New_uniprot_stop'].astype(int) - pfam[(pfam['PDB ID']==pdb) & (pfam['PDB chain ID']==chain)]['New_PDB_stop'].astype(int)
                    # <- idáig jó - inentől nem -->
                    for (resname,resid) in pdbseq:
                        try:
                            s=table.loc[(gene ,'nums')]==str(resid+increment)
                            col=s[s].index[0]
                            table.loc[(gene, pdb_id),col]=resname
                        except KeyError:
                            continue

table.to_csv('../data/table_from_alignment_with_pdbs.csv')parser = PDB.PDBParser(QUIET=True)
x = 0

for cat in pfam["Category"].unique():
    for gene in pfam[pfam["Category"] == cat]["Gene name"].unique():
        for pdb in pfam[pfam["Gene name"] == gene]["PDB ID"].unique():
            for chain in pfam[pfam["PDB ID"] == pdb]["PDB chain ID"].unique():
                for pole in pfam[(pfam["PDB ID"] == pdb) & (pfam["PDB chain ID"] == chain)]["Pole"].unique():
                    pdb_id = str(pdb+'_'+chain+'_SH2_'+pole)
                    try:
                        #print(pdb_id)
                        structure = parser.get_structure(pdb_id, '/home/takacsg/Documents/sh2db/Structures/'+cat+'/'+gene+'/'+pdb+'/'+pdb+'_'+chain+'_SH2_'+pole+'.pdb') # struct delles pdb-fájlokat nem szereti!
                    except ValueError:
                        continue
                    #model = structure[0]
                    residues = PDB.Selection.unfold_entities(structure, "R")
                    chain = pdb_id.split('_')[1]
                    """for residue in residues:
                        if residue in ['Hoh']: #This can be expanded if multiple problematic residues occur
                            residues.remove(residue) # dropping key in the next line (for "Hoh")
                    print(residues)"""
                    try:
                        pdbseq = list(zip([protein_letters_3to1[i.get_resname().title()] for i in residues],[i.get_id()[1] for i in residues]))
                    except KeyError: #Hoh caused the Keyerror problem but was hard to remove

                        try:
                            s=table.loc[(gene ,'nums')]==str(resid+increment)
                            col=s[s].index[0]
                            table.loc[(gene, pdb_id),col]=resname
                        except KeyError:
                            continue

table.to_csv('../data/table_from_alignment_with_pdbs.csv')parser = PDB.PDBParser(QUIET=True)
x = 0

for cat in pfam["Category"].unique():
    for gene in pfam[pfam["Category"] == cat]["Gene name"].unique():
        for pdb in pfam[pfam["Gene name"] == gene]["PDB ID"].unique():
                   # <- idáig jó - inentől nem -->

x = 0

for cat in pfam["Category"].unique():
    for gene in pfam[pfam["Category"] == cat]["Gene name"].unique():
        for pdb in pfam[pfam["Gene name"] == gene]["PDB ID"].unique():
            for chain in pfam[pfam["PDB ID"] == pdb]["PDB chain ID"].unique():
                for pole in pfam[(pfam["PDB ID"] == pdb) & (pfam["PDB chain ID"] == chain)]["Pole"].unique():
                    pdb_id = str(pdb+'_'+chain+'_SH2_'+pole)
                    try:
                  # <- idáig jó - inentől nem -->

x = 0

for cat in pfam["Category"].unique():
    for gene in pfam[pfam["Category"] == cat]["Gene name"].unique():
        for pdb in pfam[pfam["Gene name"] == gene]["PDB ID"].unique():
            for chain in pfam[pfam["PDB ID"] == pdb]["PDB chain ID"].unique():
                for pole in pfam[(pfam["PDB ID"] == pdb) & (pfam["PDB chain ID"] == chain)]["Pole"].unique():
                    pdb_id = str(pdb+'_'+chain+'_SH2_'+pole)
                    try:
                        #print(pdb_id)
                        structure = parser.get_structure(pdb_id, '/home/takacsg/Documents/sh2db/Structures/'+cat+'/'+gene+'/'+pdb+'/'+pdb+'_'+chain+'_SH2_'+pole+'.pdb') # struct delles pdb-fájlokat nem szereti!
                    except ValueError:
                        continue
                    #model = structure[0]
                    residues = PDB.Selection.unfold_entities(structure, "R")
                    chain = pdb_id.split('_')[1]
                    """for residue in residues:
                        if residue in ['Hoh']: #This can be expanded if multiple problematic residues occur
                            residues.remove(residue) # dropping key in the next line (for "Hoh")
                    print(residues)"""
                    try:
                        pdbseq = list(zip([protein_letters_3to1[i.get_resname().title()] for i in residues],[i.get_id()[1] for i in residues]))
                    except KeyError: #Hoh caused the Keyerror problem but was hard to remove
                        continue
                    if x % 1 == 0:
                        print("PDB ID", pdb_id, "\n", "CHAIN", chain, "\n", "PDBSEQ", pdbseq)
                    increment = int(pfam[(pfam['PDB ID']==pdb) & (pfam['PDB chain ID']==chain)]['New_uniprot_stop'] & (pfam["Pole"] == pole)) - int(pfam[(pfam['PDB ID']==pdb) & (pfam['PDB chain ID']==chain)]['New_PDB_stop'] & (pfam["Pole"] == pole))
                    # <- idáig jó - inentől nem -->
                    for (resname,resid) in pdbseq:
                        try:
                            s=table.loc[(gene ,'nums')]==str(resid+increment)
                            col=s[s].index[0]
                            table.loc[(gene, pdb_id),col]=resname
                        except KeyError:
                            continue

table.to_csv('../data/table_from_alignment_with_pdbs.csv')

In [ ]:
# Calculating the difference between the new uniprot stop and new pdb stop points:
for cat in pfam["Category"].unique():
    for gene in pfam[pfam["Category"] == cat]["Gene name"].unique():
        for pdb in pfam[pfam["Gene name"] == gene]["PDB ID"].unique():
            for chain in pfam[pfam["PDB ID"] == pdb]["PDB chain ID"].unique():
                for pole in pfam[(pfam["PDB ID"] == pdb) & (pfam["PDB chain ID"] == chain)]["Pole"].unique():
            increment = int(pfam[(pfam['PDB ID']==pdb) & (pfam['PDB chain ID']==chain)]['New_uniprot_stop']) - pfam[(pfam['PDB ID']==pdb) & (pfam['PDB chain ID']==chain)]['New_PDB_stop'])

In [ ]:
increment = int( pfam[(pfam['PDB ID']==pdb) & (pfam['PDB chain ID']==chain)]['New_uniprot_stop']) - pfam[(pfam['PDB ID']==pdb) & (pfam['PDB chain ID']==chain)]['New_PDB_stop'])

In [18]:
table.to_csv('../data/table_from_alignment_with_pdbs.csv')
